In [1]:
!pip install xlsxwriter

In [2]:
# Necessary Imports
import os
import os.path
import urllib
from urllib import request
import numpy as np
import pandas as pd
import xlsxwriter
from matplotlib import pyplot as plt

In [3]:
# Download Data Files if Not Currently Downloaded into the Current Working Directory
DOWNLOAD_DIR = "/content/"

# Set Needed File Names
needed_files = ["nba odds 2007-08.xlsx", "nba odds 2008-09.xlsx",
                "nba odds 2009-10.xlsx", "nba odds 2010-11.xlsx",
                "nba odds 2011-12.xlsx", "nba odds 2012-13.xlsx",
                "nba odds 2013-14.xlsx", "nba odds 2014-15.xlsx",
                "nba odds 2015-16.xlsx", "nba odds 2016-17.xlsx",
                "nba odds 2017-18.xlsx", "nba odds 2018-19.xlsx",
                "nba odds 2019-20.xlsx", "nba odds 2020-21.xlsx"]

# Remove Space from Needed File Names
file_urls = []
for filename in needed_files:
  file_urls.append(filename.replace(' ', '%20'))
  
# Create Season Data List
season_data = []

# Read and Download Needed Files
for i in range(len(needed_files)):
    if not os.path.isfile(DOWNLOAD_DIR + needed_files[i]):
      DOWNLOAD_ROOT = "https://raw.githubusercontent.com/LMastro99/Sports-Betting-Module/main/season-data/raw-data/"
      url = DOWNLOAD_ROOT + file_urls[i]
      urllib.request.urlretrieve(url, DOWNLOAD_DIR + needed_files[i])
    print("Downloaded", needed_files[i])    

    # Create Dataframe "Layers" with the Data From the XLSX Files
    download = pd.read_excel(DOWNLOAD_DIR + needed_files[i])
    season_data.append(download)

# Convert Season Data to Numpy Array of DataFrames
season_data = np.array(season_data, dtype = 'object')

Downloaded nba odds 2007-08.xlsx
Downloaded nba odds 2008-09.xlsx
Downloaded nba odds 2009-10.xlsx
Downloaded nba odds 2010-11.xlsx
Downloaded nba odds 2011-12.xlsx
Downloaded nba odds 2012-13.xlsx
Downloaded nba odds 2013-14.xlsx
Downloaded nba odds 2014-15.xlsx
Downloaded nba odds 2015-16.xlsx
Downloaded nba odds 2016-17.xlsx
Downloaded nba odds 2017-18.xlsx
Downloaded nba odds 2018-19.xlsx
Downloaded nba odds 2019-20.xlsx
Downloaded nba odds 2020-21.xlsx


In [4]:
# Correct Date Column
for i in range(len(season_data)):
  season_data[i]['Date'] = season_data[i]['Date'].astype('object')
  ymod = 1
  for r in range(len(season_data[i])):
    date = season_data[i]['Date'][r]
    month = str (date // 100)
    day = str (date % 100)
    if (month == '1' and int (day) >= 1):
      ymod = 0
    year = str (2000 + int (needed_files[i][-7:-5]) - ymod)
    date = month + '/' + day + '/' + year
    season_data[i]['Date'][r] = date
  season_data[i]['Date'] = pd.to_datetime(season_data[i]['Date'], format = '%m/%d/%Y')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [5]:
# View All Season Data Info
for s in range(len(season_data)):
  print(needed_files[s])
  print(season_data[s].info())
  print()

nba odds 2007-08.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2632 entries, 0 to 2631
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2632 non-null   datetime64[ns]
 1   Rot     2632 non-null   int64         
 2   VH      2632 non-null   object        
 3   Team    2632 non-null   object        
 4   1st     2632 non-null   int64         
 5   2nd     2632 non-null   int64         
 6   3rd     2632 non-null   int64         
 7   4th     2632 non-null   int64         
 8   Final   2632 non-null   int64         
 9   Open    2632 non-null   object        
 10  Close   2632 non-null   object        
 11  ML      2632 non-null   object        
 12  2H      2632 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(6)
memory usage: 267.4+ KB
None

nba odds 2008-09.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2630 entries, 0 to 2629
Data columns (total 13 columns):
 #   

In [6]:
# Drop ['Rot'], ['Open'], and ['2H'] Columns from each Season's Data
for s in range(len(season_data)):
  season_data[s].drop(columns = ['Rot', 'Open', 'Close', '2H'], inplace = True)

In [7]:
# Rename ['VH'] to ['Home'] for each Season
for s in range(len(season_data)):
  season_data[s].rename(columns = {'VH' : 'Home'}, inplace = True)

In [8]:
# View All Season Data Info
for s in range(len(season_data)):
  print(needed_files[s])
  print(season_data[s].info())
  print()

nba odds 2007-08.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2632 entries, 0 to 2631
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2632 non-null   datetime64[ns]
 1   Home    2632 non-null   object        
 2   Team    2632 non-null   object        
 3   1st     2632 non-null   int64         
 4   2nd     2632 non-null   int64         
 5   3rd     2632 non-null   int64         
 6   4th     2632 non-null   int64         
 7   Final   2632 non-null   int64         
 8   ML      2632 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 185.2+ KB
None

nba odds 2008-09.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2630 entries, 0 to 2629
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2630 non-null   datetime64[ns]
 1   Home    2630 non-null   object        
 2   Te

In [9]:
# Drop Neutral Games to a Separate DataFrame "Layer"
neutral_games_list = []
for s in range(len(season_data)):
  current_season = season_data[s]
  neutral_games = pd.DataFrame(current_season[current_season['Home'] == 'N'])
  num_neutral_games = len(pd.DataFrame(neutral_games))
  if (num_neutral_games > 0):
    season_data[s] = current_season.append(neutral_games).drop_duplicates(keep=False)
  neutral_games_list.append(neutral_games)
  
neutral_games_list

[Empty DataFrame
 Columns: [Date, Home, Team, 1st, 2nd, 3rd, 4th, Final, ML]
 Index: [], Empty DataFrame
 Columns: [Date, Home, Team, 1st, 2nd, 3rd, 4th, Final, ML]
 Index: [], Empty DataFrame
 Columns: [Date, Home, Team, 1st, 2nd, 3rd, 4th, Final, ML]
 Index: [],            Date Home       Team  1st  2nd  3rd  4th  Final   ML
 1822 2011-03-04    N    Toronto   23   28   30   22    103  125
 1823 2011-03-04    N  NewJersey   27   20   31   38    116 -145
 1844 2011-03-05    N    Toronto   30   32   26   22    136  110
 1845 2011-03-05    N  NewJersey   23   42   23   22    137 -130, Empty DataFrame
 Columns: [Date, Home, Team, 1st, 2nd, 3rd, 4th, Final, ML]
 Index: [],            Date Home     Team  1st  2nd  3rd  4th  Final   ML
 1156 2013-01-17    N  NewYork   29   27   19   27    102 -230
 1157 2013-01-17    N  Detroit   17   24   22   24     87  190,            Date Home      Team  1st  2nd  3rd  4th  Final   ML
 1152 2014-01-16    N  Brooklyn   31   34   34   28    127  100
 1153 

In [10]:
season_data[-2]

,Date,Home,Team,1st,2nd,3rd,4th,Final,ML
0,2019-10-22,V,NewOrleans,30,31,25,31,122,230
1,2019-10-22,H,Toronto,27,29,32,29,130,-280
2,2019-10-22,V,LALakers,25,29,31,17,102,-180
3,2019-10-22,H,LAClippers,22,40,23,27,112,150
4,2019-10-23,V,Detroit,27,27,29,36,119,240
...,...,...,...,...,...,...,...,...,...
1937,2020-03-11,H,Philadelphia,34,36,29,25,124,-800
1938,2020-03-11,V,Charlotte,22,37,28,22,109,500
1939,2020-03-11,H,Miami,40,11,26,21,98,-700
1940,2020-03-11,V,Denver,29,21,35,12,97,-140


In [11]:
neutral_games_list[-2]

,Date,Home,Team,1st,2nd,3rd,4th,Final,ML
1942,2020-07-30,N,Utah,23,25,31,27,106,125
1943,2020-07-30,N,NewOrleans,26,34,27,17,104,-145
1944,2020-07-30,N,LAClippers,23,29,25,24,101,165
1945,2020-07-30,N,LALakers,35,19,22,27,103,-185
1946,2020-07-31,N,Orlando,36,34,41,17,128,-340
...,...,...,...,...,...,...,...,...,...
2281,2020-10-06,N,Miami,22,25,23,26,96,270
2282,2020-10-09,N,Miami,25,35,28,23,111,280
2283,2020-10-09,N,LALakers,24,32,26,26,108,-340
2284,2020-10-11,N,LALakers,28,36,23,19,106,-250


In [12]:
# Determine Favorite and Underdog
fav_games_list = []
und_games_list = []
for s in range(len(season_data)):
  current_season = season_data[s]
  neutral_games = pd.DataFrame(current_season[current_season['Home'] == 'N'])
  num_neutral_games = len(pd.DataFrame(neutral_games))
  if (num_neutral_games > 0):
    neutral_games_list.append(neutral_games)
neutral_games_list

[Empty DataFrame
 Columns: [Date, Home, Team, 1st, 2nd, 3rd, 4th, Final, ML]
 Index: [], Empty DataFrame
 Columns: [Date, Home, Team, 1st, 2nd, 3rd, 4th, Final, ML]
 Index: [], Empty DataFrame
 Columns: [Date, Home, Team, 1st, 2nd, 3rd, 4th, Final, ML]
 Index: [],            Date Home       Team  1st  2nd  3rd  4th  Final   ML
 1822 2011-03-04    N    Toronto   23   28   30   22    103  125
 1823 2011-03-04    N  NewJersey   27   20   31   38    116 -145
 1844 2011-03-05    N    Toronto   30   32   26   22    136  110
 1845 2011-03-05    N  NewJersey   23   42   23   22    137 -130, Empty DataFrame
 Columns: [Date, Home, Team, 1st, 2nd, 3rd, 4th, Final, ML]
 Index: [],            Date Home     Team  1st  2nd  3rd  4th  Final   ML
 1156 2013-01-17    N  NewYork   29   27   19   27    102 -230
 1157 2013-01-17    N  Detroit   17   24   22   24     87  190,            Date Home      Team  1st  2nd  3rd  4th  Final   ML
 1152 2014-01-16    N  Brooklyn   31   34   34   28    127  100
 1153 

In [15]:
# Export Neutral Games Data
writer = pd.ExcelWriter('neutral games.xlsx', engine='xlsxwriter')
for n in range(len(neutral_games_list)):
  year = str (2007 + n) + '-' + str (8 + n)
  neutral_games_list[n].to_excel(writer, sheet_name = year)
writer.save()

In [16]:
# Export Home-Away Games Data
writer = pd.ExcelWriter('home-away games.xlsx', engine='xlsxwriter')
for s in range(len(season_data)):
  year = str (2007 + s) + '-' + str (8 + s)
  season_data[s].to_excel(writer, sheet_name = year)
writer.save()